In [1]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import random
import pickle
import datetime
import os
import torch.nn.functional as F
from scipy.stats import special_ortho_group
from scipy.spatial.transform import Rotation as R
import torch.nn.functional as F
#from pytorch_metric_learning import miners, losses
import wandb
import itertools

from torch.nn.functional import softmax
import open3d  as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('test_subset_random.pkl', 'rb') as file:
    loaded_val_list = pickle.load(file)
with open('correctly_predicted_indices.pkl', 'rb') as file:
    indeces_corrects = pickle.load(file)

with open('incorrectly_predicted_indices.pkl', 'rb') as file:
    indeces_INcorrects = pickle.load(file)

with open('preds_0_indices.pkl', 'rb') as file:
    preds_0 = pickle.load(file)

with open('preds_1_indices.pkl', 'rb') as file:
    preds_1 = pickle.load(file)

In [3]:
def translate_to_origin(frag):
    """ This function translate each fragment in the origin
    """
    
    frag[:,:3] -= torch.mean(frag[:,:3]) 
    return frag

def normalize_points(tensor):
        tensor[:,:3] -= torch.mean(tensor[:,:3]) 
        tensor[:,:3] /= torch.max(torch.norm(tensor[:,:3], dim=1))
        return tensor

def random_rotation(frag):
    randrot = (torch.rand(3)*360).tolist()
    r = R.from_euler('zyx', randrot, degrees=True)
    frag[:,:3] = torch.from_numpy(r.apply(frag[:,:3]))
    frag[:,3:6] = torch.from_numpy(r.apply(frag[:,3:6]))
    return frag

def center_in_origin(frag):
    min_vals, _ = torch.min(frag[:, 0:3], axis=0)
    max_vals, _ = torch.max(frag[:, 0:3], axis=0)
    frag[:, 0:3] = (frag[:, 0:3] - min_vals) / (max_vals - min_vals)
    
    return frag

In [33]:
def visualize_point_clouds(index, indeces_corrects):
    
    a = loaded_val_list[index][0]
    b = loaded_val_list[index][1]
    color = loaded_val_list[index][2]
    a = torch.from_numpy(a)
    b = torch.from_numpy(b)

    a= translate_to_origin(a)
    b= translate_to_origin(b)
    a =random_rotation(a)
    b=random_rotation(b)
    # Extract the first three columns for XYZ coordinates
    a_xyz_coordinates = a[:, :3]
    b_xyz_coordinates = b[:, :3]

    # Create PointCloud objects for both arrays
    point_cloud_a = o3d.geometry.PointCloud()
    point_cloud_a.points = o3d.utility.Vector3dVector(a_xyz_coordinates)

    point_cloud_b = o3d.geometry.PointCloud()
    point_cloud_b.points = o3d.utility.Vector3dVector(b_xyz_coordinates)

    # Create a visualizer and add both point clouds
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='a')
    vis.add_geometry(point_cloud_a)
    vis.add_geometry(point_cloud_b)
    intro_string="Couple"+str(index)
    print(f"index: {index}, color: {color}")
    # Print the title
    if (sum(indeces_corrects == index) == 1) and (loaded_val_list[index][2] == 1):
        vis.create_window(window_name=str(intro_string) + ' Y_true=1, Y_preds=1')

    elif (sum(indeces_corrects == index) == 0) and (loaded_val_list[index][2] == 1):
        vis.create_window(window_name=str(intro_string) + ' Y_true=1, Y_preds=0')

    elif (sum(indeces_corrects == index) == 1) and (loaded_val_list[index][2] == 0):
        vis.create_window(window_name=str(intro_string) + ' Y_true=0, Y_preds=0')

    elif (sum(indeces_corrects == index) == 0) and (loaded_val_list[index][2] == 0):
        vis.create_window(window_name=str(intro_string) + ' Y_true=0, Y_preds=1')

    else:
        
        pass    
    
    opt = vis.get_render_option()
    opt.point_size = 5 

    vis.run()
    vis.destroy_window()

In [33]:
import open3d as o3d
import torch

def visualize_point_clouds_spacing(index, indeces_corrects, spacing=1):
    a = loaded_val_list[index][0]
    b = loaded_val_list[index][1]
    color = loaded_val_list[index][2]
    a = torch.from_numpy(a)
    b = torch.from_numpy(b)

    a = translate_to_origin(a)
    b = translate_to_origin(b)
    a = random_rotation(a)
    b = random_rotation(b)

    # Extract the first three columns for XYZ coordinates
    a_xyz_coordinates = a[:, :3]
    b_xyz_coordinates = b[:, :3]

    # Create PointCloud objects for both arrays
    point_cloud_a = o3d.geometry.PointCloud()
    point_cloud_a.points = o3d.utility.Vector3dVector(a_xyz_coordinates)

    point_cloud_b = o3d.geometry.PointCloud()
    point_cloud_b.points = o3d.utility.Vector3dVector(b_xyz_coordinates)

    # Translate the second point cloud along the X-axis
    b_xyz_coordinates[:, 2] += spacing
    point_cloud_b.points = o3d.utility.Vector3dVector(b_xyz_coordinates)

    # Create a visualizer and add both point clouds
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='a')
    vis.add_geometry(point_cloud_a)
    vis.add_geometry(point_cloud_b)

    intro_string = "Couple" + str(index)
    print(f"index: {index}, color: {color}")

    # Print the title
    if (sum(indeces_corrects == index) == 1) and (loaded_val_list[index][2] == 1):
        vis.create_window(window_name=str(intro_string) + ' Y_true=1, Y_preds=1')

    elif (sum(indeces_corrects == index) == 0) and (loaded_val_list[index][2] == 1):
        vis.create_window(window_name=str(intro_string) + ' Y_true=1, Y_preds=0')

    elif (sum(indeces_corrects == index) == 1) and (loaded_val_list[index][2] == 0):
        vis.create_window(window_name=str(intro_string) + ' Y_true=0, Y_preds=0')

    elif (sum(indeces_corrects == index) == 0) and (loaded_val_list[index][2] == 0):
        vis.create_window(window_name=str(intro_string) + ' Y_true=0, Y_preds=1')

    else:
        pass

    opt = vis.get_render_option()
    opt.point_size = 5

    vis.run()
    vis.destroy_window()


In [6]:
import open3d as o3d
import torch
import numpy as np

def visualize_point_clouds_colored(index, indeces_corrects):
    
    b = loaded_val_list[index][0]
    a = loaded_val_list[index][1]
    color = loaded_val_list[index][2]
    a = torch.from_numpy(a)
    b = torch.from_numpy(b)
    
    # Extract the first three columns for XYZ coordinates
    a_xyz_coordinates = a[:, :3]
    b_xyz_coordinates = b[:, :3]

    # Create PointCloud objects for both arrays
    point_cloud_a = o3d.geometry.PointCloud()
    point_cloud_a.points = o3d.utility.Vector3dVector(a_xyz_coordinates)

    point_cloud_b = o3d.geometry.PointCloud()
    point_cloud_b.points = o3d.utility.Vector3dVector(b_xyz_coordinates)
    point_cloud_b.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 0], (b_xyz_coordinates.shape[0], 1)))  # Colore nero

    # Create a visualizer and add both point clouds
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='a')
    vis.add_geometry(point_cloud_a)
    vis.add_geometry(point_cloud_b)
    
    intro_string = "Couple" + str(index)
    print(f"index: {index}, color: {color}")

    # Print the title based on conditions (modify as needed)
    if (sum(indeces_corrects == index) == 1) and (loaded_val_list[index][2] == 1):
        vis.create_window(window_name=str(intro_string) + ' Y_true=1, Y_preds=1')

    elif (sum(indeces_corrects == index) == 0) and (loaded_val_list[index][2] == 1):
        vis.create_window(window_name=str(intro_string) + ' Y_true=1, Y_preds=0')

    elif (sum(indeces_corrects == index) == 1) and (loaded_val_list[index][2] == 0):
        vis.create_window(window_name=str(intro_string) + ' Y_true=0, Y_preds=0')

    elif (sum(indeces_corrects == index) == 0) and (loaded_val_list[index][2] == 0):
        vis.create_window(window_name=str(intro_string) + ' Y_true=0, Y_preds=1')

    else:
        pass    
    
    opt = vis.get_render_option()
    opt.point_size = 5 

    vis.run()
    vis.destroy_window()


In [105]:
visualize_point_clouds(4000,indeces_corrects)

index: 4000, color: 0


In [21]:
import open3d as o3d
import numpy as np
def visualize_single_pointcloud(couple,element,rotate):
    # Supponiamo che il tuo array sia chiamato "point_cloud_array"
    # Estrai le prime tre colonne per le coordinate XYZ
    tensor = loaded_val_list[couple][element]
    if rotate ==1:
     tensor = random_rotation(tensor)
    else:
        tensor = tensor
         
    xyz_coordinates = tensor[:, :3]
    
    # Crea un oggetto PointCloud di Open3D
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(xyz_coordinates)



    # Crea una visualizzazione e aggiungi il point cloud
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(point_cloud)
    opt = vis.get_render_option()
    opt.point_size = 8
    # Visualizzazione interattiva
    vis.run()
    vis.destroy_window()

In [31]:
visualize_single_pointcloud(1,0,1)